In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
# Setup device agnostic code
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [17]:
# Loading breast cancer dataset
data= load_breast_cancer()
X= data.data
Y= data.target

In [18]:
# Spliting dataset into training and testing sets
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# Converting numpy array to tensors
X_train= torch.tensor(X_train, dtype=torch.float32).to(device)
X_test= torch.tensor(X_test, dtype=torch.float32).to(device)
Y_train= torch.tensor(Y_train, dtype=torch.float32).to(device)
Y_test= torch.tensor(Y_test, dtype=torch.float32).to(device)

In [21]:
# Neural Network
class BreastCancer(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(BreastCancer, self).__init__()
    self.fc1= nn.Linear(input_size, hidden_size)
    self.relu= nn.ReLU()
    self.fc2= nn.Linear(hidden_size, output_size)
    self.sigmoid= nn.Sigmoid()

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    out= self.fc1(x)
    out= self.relu(out)
    out= self.fc2(out)
    out= self.sigmoid(out)
    return out


In [22]:
# Defining hyperparameters
input_size= X_train.shape[1]
hidden_size= 64
output_size= 1
learning_rate= 0.001
epochs=100

In [23]:
model1= BreastCancer(input_size, hidden_size, output_size).to(device)

In [24]:
# Defining loss function and optimizer
lossfunc= nn.BCELoss()
optimizer= optim.Adam(model1.parameters(),
                      lr= learning_rate)

In [25]:
Y_train.size(), X_train.size(), Y_train.view(-1,1).size()

(torch.Size([455]), torch.Size([455, 30]), torch.Size([455, 1]))

In [28]:
# Training Loop
for epoch in range(epochs):
  model1.train()
  optimizer.zero_grad
  output= model1(X_train)
  loss= lossfunc(output, Y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  with torch.inference_mode():
    predicted= output.round()
    correct= (predicted == Y_train.view(-1,1)).float().sum()
    accuracy= correct/Y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%")

Epoch [10/100], Loss: 0.1014, Accuracy: 96.48%
Epoch [20/100], Loss: 0.0968, Accuracy: 97.14%
Epoch [30/100], Loss: 0.0927, Accuracy: 97.14%
Epoch [40/100], Loss: 0.0897, Accuracy: 97.14%
Epoch [50/100], Loss: 0.0885, Accuracy: 97.36%
Epoch [60/100], Loss: 0.0894, Accuracy: 97.36%
Epoch [70/100], Loss: 0.0914, Accuracy: 97.58%
Epoch [80/100], Loss: 0.0930, Accuracy: 97.58%
Epoch [90/100], Loss: 0.0934, Accuracy: 97.58%
Epoch [100/100], Loss: 0.0923, Accuracy: 97.58%


In [31]:
# Evaluating on training set
model1.eval()
with torch.inference_mode():
  output= model1(X_train)
  predicted= output.round()
  correct= (predicted == Y_train.view(-1,1)).float().sum()
  accuracy= correct/Y_train.size(0)
  print(f"Training Accuracy: {accuracy.item()*100:.2f}%")

Training Accuracy: 97.58%


In [32]:
# Evaluating on testing set
model1.eval()
with torch.inference_mode():
  output= model1(X_test)
  predicted= output.round()
  correct= (predicted == Y_test.view(-1,1)).float().sum()
  accuracy= correct/Y_test.size(0)
  print(f"Testing Accuracy: {accuracy.item()*100:.2f}%")

Testing Accuracy: 97.37%
